In [ ]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 348.9MB 24.7MB/s ta 0:00:011   2% |▉                               | 8.9MB 94.8MB/s eta 0:00:04           | 10.9MB 95.1MB/s eta 0:00:0404                         | 25.5MB 92.3MB/s eta 0:00:04                         | 30.3MB 92.8MB/s eta 0:00:04.8MB 92.3MB/s eta 0:00:04.5MB/s eta 0:00:04��███                            | 44.0MB 93.0MB/s eta 0:00:04��█▌                           | 48.9MB 90.0MB/s eta 0:00:04                   | 53.6MB 92.9MB/s eta 0:00:04██▍                          | 58.5MB 94.3MB/s eta 0:00:04��▊                          | 62.5MB 88.1MB/s eta 0:00:04██▏                         | 67.5MB 93.9MB/s eta 0:00:033MB 96.3MB/s eta 0:00:031MB 83.4MB/s eta 0:00:04                     | 81.4MB 96.0MB/s eta 0:00:03��████▉                        | 85.7MB 94.9MB/s eta 0:00:039MB 93.2MB/s eta 0:00:035MB 96.1MB/s eta 0:00:035MB 92.9MB/s eta 0:00:03% |█████████▍                      | 102.6MB 85.2MB/s eta 0:00:03   30% |█████████▉            

    100% |████████████████████████████████| 501kB 99.5MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 83.9MB/s ta 0:00:01
    35% |███████████▎                    | 9.3MB 94.8MB/s eta 0:00:011

In [3]:
!pip uninstall -y PyWavelets
!pip install PyWavelets

Uninstalling PyWavelets-0.5.2:
  Successfully uninstalled PyWavelets-0.5.2
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 4.4MB 103.6MB/s ta 0:00:01██▍| 4.3MB 112.3MB/s eta 0:00:01
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config
from PIL import Image

import math
from random import randrange

import pandas as pd

import matplotlib.pyplot as plt

from skimage import img_as_float
from skimage.measure import compare_ssim as ssim, compare_psnr as psnr

import json

In [20]:
def evaluate(checkpoint_dir, model, valid_lr_img, valid_hr_img, image_name, G = None, save_dir = "validation-samples"):

    os.makedirs(save_dir, exist_ok=True)
    
    valid_lr_img = (valid_lr_img / 127.5) - 1  # rescale to ［－1, 1]

    if not G:
        G = get_G([1, None, None, 3])
        G.load_weights(os.path.join(checkpoint_dir, model))
        G.eval()

    valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
    valid_lr_img = valid_lr_img[np.newaxis,:,:,:]
    size = [valid_lr_img.shape[1], valid_lr_img.shape[2]]

    out = G(valid_lr_img).numpy()
    
    model_num = model.replace(".h5","").split("-")[1]

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))  # LR size: (339, 510, 3) /  gen HR size: (1, 1356, 2040, 3)
    
    if not os.path.isfile('sr-' + model_num + "-" + image_name):
        tl.vis.save_image(out[0], os.path.join(save_dir, 'sr-' + model_num + "-" + image_name))

        out_bicu = scipy.misc.imresize(valid_lr_img[0], [size[0] * 4, size[1] * 4], interp='bicubic', mode=None)
        tl.vis.save_image(out_bicu, os.path.join(save_dir, 'bic-' + model_num + "-" + image_name))

    sr_smaller = tf.image.resize(out[0], size=size)
    hr_smaller = tf.image.resize(valid_hr_img, size=size)

    validate = {
        "sr" : out[0],
        "sr_resized" : sr_smaller.numpy(),
        
        "lr" : valid_lr_img[0],
        "bic" : out_bicu,
        
        "hr" : valid_hr_img, 
        "hr_resized" : hr_smaller.numpy(),
    }
    
    data = {
        "G" : G,
        
        "model" : model,

        "psnr_lr" : psnr( validate.get("lr"),  validate.get("sr_resized")),
        "ssim_lr" : ssim(validate.get("lr"),  validate.get("sr_resized"), multichannel=True),

        "psnr_hr_4" : psnr( validate.get("hr_resized"),  validate.get("sr_resized"), data_range = 255),
        "ssim_hr_4" : ssim(validate.get("hr_resized"),  validate.get("sr_resized"), multichannel=True),
        
        "psnr_hr" : psnr( validate.get("hr"),  validate.get("sr")),
        "ssim_hr" : ssim(validate.get("hr"),  validate.get("sr"), multichannel=True),

        "psnr_bic_hr" : psnr( validate.get("hr"),  validate.get("bic")),
        "ssim_bic_hr" : ssim( validate.get("hr"),  validate.get("bic"), multichannel=True),
    }
    return data
       

In [32]:
def evaluate_downsample(checkpoint_dir, model, valid_hr_img, image_name, G = None, save_dir = "validation-ds-samples"):

    os.makedirs(save_dir, exist_ok=True)
    
    size = [int(valid_hr_img.shape[0]/4), int(valid_hr_img.shape[1]/4)]
    
    hr_smaller = tf.image.resize(valid_hr_img, size=size)
    
    valid_lr_img = (hr_smaller / 127.5) - 1  # rescale to ［－1, 1]

    if not G:
        G = get_G([1, None, None, 3])
        G.load_weights(os.path.join(checkpoint_dir, model))
        G.eval()

    valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
    valid_lr_img = valid_lr_img[np.newaxis,:,:,:]
    

    out = G(valid_lr_img).numpy()
    
    model_num = model.replace(".h5","").split("-")[1]

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))  # LR size: (339, 510, 3) /  gen HR size: (1, 1356, 2040, 3)
    
    if not os.path.isfile('sr-' + model_num + "-" + image_name):
        tl.vis.save_image(out[0], os.path.join(save_dir, 'sr-' + model_num + "-" + image_name))

        out_bicu = scipy.misc.imresize(valid_lr_img[0], [size[0] * 4, size[1] * 4], interp='bicubic', mode=None)
        tl.vis.save_image(out_bicu, os.path.join(save_dir, 'bic-' + model_num + "-" + image_name))

    sr_smaller = tf.image.resize(out[0], size=size)
    

    validate = {
        "sr" : out[0],
        "sr_resized" : sr_smaller.numpy(),
        
        "lr" : valid_lr_img[0],
        "bic" : out_bicu,
        
        "hr" : valid_hr_img, 
        "hr_resized" : hr_smaller.numpy(),
    }
    
    data = {
#         "G" : G,
        
        "model" : model,

        "psnr_lr" : psnr( validate.get("lr"),  validate.get("sr_resized")),
        "ssim_lr" : ssim(validate.get("lr"),  validate.get("sr_resized"), multichannel=True),

        "psnr_hr_4" : psnr( validate.get("hr_resized"),  validate.get("sr_resized"), data_range = 255),
        "ssim_hr_4" : ssim(validate.get("hr_resized"),  validate.get("sr_resized"), multichannel=True),
        
        "psnr_hr" : psnr( validate.get("hr"),  validate.get("sr")),
        "ssim_hr" : ssim(validate.get("hr"),  validate.get("sr"), multichannel=True),

        "psnr_bic_hr" : psnr( validate.get("hr"),  validate.get("bic")),
        "ssim_bic_hr" : ssim( validate.get("hr"),  validate.get("bic"), multichannel=True),
    }
    return data
       

In [6]:
###====================== PRE-LOAD DATA ===========================###
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))

valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, path=config.VALID.lr_img_path, n_threads=32)

valid_hr_imgs = tl.vis.read_images(valid_hr_img_list, path=config.VALID.hr_img_path, n_threads=32)

[TL] read 32 from DIV2K/DIV2K_valid_LR_difficult/
[TL] read 64 from DIV2K/DIV2K_valid_LR_difficult/
[TL] read 96 from DIV2K/DIV2K_valid_LR_difficult/
[TL] read 100 from DIV2K/DIV2K_valid_LR_difficult/
[TL] read 32 from DIV2K/DIV2K_valid_HR/
[TL] read 64 from DIV2K/DIV2K_valid_HR/
[TL] read 96 from DIV2K/DIV2K_valid_HR/
[TL] read 100 from DIV2K/DIV2K_valid_HR/


In [7]:
def createPyPlot(validate_data, resized = True):


    label = 'SSIM: {:.2f}, sk_psnr:{:.2f} PSNR: {:.2f}'

    if resized: # show the images at size == the size of the input LR image
        fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(17, 12),
                             sharex=True, sharey=True)
        ax = axes.ravel()
        
        ax[0].imshow(validate_data.get("images").get("lr"))
        ax[0].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[0].set_title('valid LR image')
        
        ax[1].imshow(validate_data.get("images").get("sr_resized"))
        ax[1].set_xlabel(label.format(validate_data.get("ssim_lr"), validate_data.get("psnr_lr"), validate_data.get("PSNR_lr")))
        ax[1].set_title('generated image resized *-4 vs LR image')
        
        ax[2].imshow(validate_data.get("images").get("hr_resized"))
        ax[2].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[2].set_title('valid HR resized *-4')      
        
        ax[3].imshow(validate_data.get("images").get("sr_resized"))
        ax[3].set_xlabel(label.format(validate_data.get("ssim_hr_4"), validate_data.get("psnr_hr_4"), validate_data.get("PSNR_hr_4")))
        ax[3].set_title('generated image resized *-4 vs HR resized')
        
    else: 
        
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(17, 12),
                             sharex=True, sharey=True)
        ax = axes.ravel()
    
        ax[0].imshow(validate_data.get("images").get("hr"))
        ax[0].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[0].set_title('valid HR image')

        ax[1].imshow(validate_data.get("images").get("bic"))
        ax[1].set_xlabel(label.format(validate_data.get("ssim_bic_hr"), validate_data.get("psnr_bic_hr"), validate_data.get("PSNR_bic_hr")))
        ax[1].set_title('bicubic interpolation *4 vs HR')

        ax[2].imshow(validate_data.get("images").get("sr"))
        ax[2].set_xlabel(label.format(validate_data.get("ssim_hr"), validate_data.get("psnr_hr"), validate_data.get("PSNR_bic_hr")))
        ax[2].set_title('generated image vs HR')
    
    plt.tight_layout()
    plt.show()

In [9]:
def compare_models_names(a):
    return int(a.replace(".h5","").split("-")[1])

In [17]:
def rand_three(l):
    return [i for i in set((randrange(l), randrange(l), randrange(l), randrange(l), randrange(l)))][:3]

rand_three(l)

[56, 33, 99]

In [18]:
models = sorted(tl.files.load_file_list(path="checkpoint", regx='g-[0-9]+\.(h5)', printable=False), key=compare_models_names)
pd.DataFrame(models).head()

,0
0,g-10.h5
1,g-20.h5
2,g-30.h5
3,g-40.h5
4,g-50.h5


In [21]:
G = None

In [26]:
l = len(valid_hr_img_list)

for image in rand_three(l):
    validate_array = []
    for model in models:
        valid_lr_img = valid_lr_imgs[image]
        valid_hr_img = valid_hr_imgs[image]
        image_name = valid_hr_img_list[image]
        
        ev = evaluate("checkpoint", model, valid_lr_img, valid_hr_img, image_name, G = G)
        
        G = ev.pop("G", G)
        validate_array.append(ev) 
        
    with open("logs/" + image_name + ".json", mode='w', encoding='utf-8') as f:
        json.dump(validate_array, f)
    

Lossy conversion from float32 to uint8. Range [-0.9872180223464966, 0.9135316610336304]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [288, 510] /  generated HR size: (1, 1152, 2040, 3)


/home/ma-user/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/home/ma-user/anaconda3/lib/python3.6/site-packages/skimage/measure/simple_metrics.py:127: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im_true.
  warn("Inputs have mismatched dtype.  Setting data_range based on "
/home/ma-user/anaconda3/lib/python3.6/site-packages/skimage/measure/_structural_similarity.py:155: UserWarning: Inputs have mismatched dtype.  Setting data_range based on X.dtype.
  warn("Inputs have mismatched dtype.  Setting data_range based on "
Lossy conversion from float32 to uint8. Range [-0.9872180223464966, 0.9135316610336304]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [288, 510] /  generated HR size: (1, 1152, 2040, 3)


Lossy conversion from float32 to uint8. Range [-0.9872180223464966, 0.9135316610336304]. Convert image to uint8 prior to saving to suppress this warning.


...

LR size: [288, 510] /  generated HR size: (1, 1152, 2040, 3)


Lossy conversion from float32 to uint8. Range [-0.9869416952133179, 0.8544871807098389]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [288, 510] /  generated HR size: (1, 1152, 2040, 3)


Lossy conversion from float32 to uint8. Range [-0.9869416952133179, 0.8544871807098389]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [288, 510] /  generated HR size: (1, 1152, 2040, 3)


In [33]:
l = len(valid_hr_img_list)

for image in rand_three(l):
    validate_ds_array = []
    for model in models:
        valid_hr_img = valid_hr_imgs[image]
        image_name = valid_hr_img_list[image]
        
        ev = evaluate_downsample("checkpoint", model, valid_hr_img, image_name, G = G)
        
#         G = ev.pop("G", G)
        validate_ds_array.append(ev) 
        
    with open("logs/" + image_name + "-ds.json", mode='w', encoding='utf-8') as f:
        json.dump(validate_ds_array, f)
    

Lossy conversion from float32 to uint8. Range [-0.9674316048622131, 0.9013864398002625]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [510, 339] /  generated HR size: (1, 2040, 1356, 3)


/home/ma-user/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:29: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/home/ma-user/anaconda3/lib/python3.6/site-packages/skimage/measure/simple_metrics.py:127: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im_true.
  warn("Inputs have mismatched dtype.  Setting data_range based on "
/home/ma-user/anaconda3/lib/python3.6/site-packages/skimage/measure/_structural_similarity.py:155: UserWarning: Inputs have mismatched dtype.  Setting data_range based on X.dtype.
  warn("Inputs have mismatched dtype.  Setting data_range based on "
Lossy conversion from float32 to uint8. Range [-0.9674316048622131, 0.9013864398002625]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [510, 339] /  generated HR size: (1, 2040, 1356, 3)


Lossy conversion from float32 to uint8. Range [-0.9674316048622131, 0.9013864398002625]. Convert image to uint8 prior to saving to suppress this warning.


...

LR size: [435, 510] /  generated HR size: (1, 1740, 2040, 3)


Lossy conversion from float32 to uint8. Range [-0.9795401692390442, 0.9754083156585693]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [435, 510] /  generated HR size: (1, 1740, 2040, 3)


Lossy conversion from float32 to uint8. Range [-0.9795401692390442, 0.9754083156585693]. Convert image to uint8 prior to saving to suppress this warning.


LR size: [435, 510] /  generated HR size: (1, 1740, 2040, 3)


In [34]:
pd.DataFrame(validate_ds_array)

,model,psnr_bic_hr,psnr_hr,psnr_hr_4,psnr_lr,ssim_bic_hr,ssim_hr,ssim_hr_4,ssim_lr
0,g-10.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
1,g-20.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
2,g-30.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
3,g-40.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
4,g-50.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
5,g-60.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
6,g-70.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
7,g-80.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
8,g-90.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715
9,g-100.h5,28.681592,4.627215,4.635659,37.235378,0.856393,-0.002236,-0.000046,0.980715


In [ ]:
createPyPlot(validate, resized = True)

In [ ]:
createPyPlot(validate, resized = False)